In [9]:
!pip install urllib2

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


In [17]:
import os
import perceval
import requests
import urllib
from bs4 import BeautifulSoup as bs
import pandas as pd 
import sys
import cv2
import os
import tqdm
import json

In [38]:
releases_url = 'https://projects.apache.org/json/foundation/releases.json?0.8321215568016254'

In [53]:
releases = pd.DataFrame(columns={'project','release','date'})

In [40]:
response = requests.get(releases_url)
response.raise_for_status()  
data = response.json()

In [54]:
for i in data.keys():
    for j in data[i].keys():
        releases = releases.append({'project':i,'release':j,'date':data[i][j]},ignore_index=True)

In [61]:
release = releases.sort_values(by=['project','date']).reset_index(drop=True)

In [62]:
release['delay'] = [0]*release.shape[0]

In [71]:
release['delay'] = pd.to_datetime(release['date']).diff()

In [84]:
release

,release,date,project,delay
0,accumulo2-maven-plugin-1.0.0,2020-07-06,accumulo,NaT
1,accumulo-1.10.1,2020-12-22,accumulo,169 days
2,accumulo-2.0.1,2020-12-25,accumulo,3 days
3,Apache.NMS-1.5.1,2020-07-03,activemq,-175 days
4,Apache.NMS-1.7.1,2020-07-03,activemq,0 days
...,...,...,...,...
3642,zeppelin-0.9.0,2020-12-20,zeppelin,150 days
3643,zeppelin-0.10.0,2021-08-17,zeppelin,240 days
3644,apache-zookeeper-3.5.9,2021-01-14,zookeeper,-215 days
3645,apache-zookeeper-3.7.0,2021-03-27,zookeeper,72 days


In [85]:
gp = release.groupby(['project']).agg({'release':lambda x: x.iloc[1:].count(),
                                 'delay':lambda x: x.iloc[1:].sum()}).reset_index()

In [88]:
gp['average_delay_in_release'] = gp['delay']/gp['release']

In [92]:
gp[['project','average_delay_in_release']].to_csv('average_delay_in_release.csv')

In [93]:
rachit = pd.read_csv('op.csv')
rachit = rachit[rachit['Is Source?']==True].reset_index(drop=True)
rachit['repo'] = rachit['Repo Name'].str.split('/').str[-1]

In [94]:
year = pd.read_csv('mail_archive_links.csv').drop({'Unnamed: 0'},axis=1)

In [95]:
year2 = pd.DataFrame(columns = {'project','mail_list_type','href','monyear','mbox_link','total_mail'})
for _,i in rachit.iterrows():
    year2 = year2.append(year[year['href'].str.contains(i.repo)],
                         ignore_index=True)

In [100]:
year2.groupby(['project'])['total_mail'].sum().reset_index().to_csv('no_of_emails.csv')